In [32]:
!pip install -q -U kaggle_environments

In [33]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [34]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [35]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [36]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[1, None]]

In [37]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

In [38]:
from kaggle_environments import make
help(make)
env = make("tictactoe")
dir(env)
help(env.reset)

Help on function make in module kaggle_environments.core:

make(environment, configuration=None, info=None, steps=None, logs=None, debug=False, state=None)
    Creates an instance of an Environment.
    
    Args:
        environment (str|Environment):
        configuration (dict, optional):
        info (dict, optional):
        steps (list, optional):
        debug (bool=False, optional): Render print() statments to stdout
        state (optional):
    
    Returns:
        Environment: Instance of a specific environment.

Help on method reset in module kaggle_environments.core:

reset(num_agents=None) method of kaggle_environments.core.Environment instance
    Resets the environment state to the initial step.
    
    Args:
        num_agents (int): Resets the state assuming a fixed number of agents.
    
    Returns:
        list of dict: The agents states after the reset.



In [39]:
%%writefile paper_agent.py

#Агент №1 играющий только Бумагу
def paper_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


In [40]:
%%writefile scissors_agent.py

#Агент №2 играющий только Ножницы
def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [41]:
%%writefile rock_random_opponent.py

#Агент №3 начинающий с камня, а затем играющий рандомно
def rock_random_opponent(observation, configuration):
    if observation.step == 0:
        return 0
    else:
        return random.randrange(0, 3)

Overwriting rock_random_opponent.py


In [42]:
%%writefile paper_random_opponent.py

#Агент №4 начинающий с бумаги, а затем играющий рандомно
def paper_random_opponent(observation, configuration):
    if observation.step == 0:
        return 0
    else:
        return random.randrange(0, 3)

Overwriting paper_random_opponent.py


In [43]:
%%writefile sciccors_random_opponent.py

#Агент №5 начинающий с ножниц, а затем играющий рандомно
def sciccors_random_opponent(observation, configuration):
    if observation.step == 0:
        return 0
    else:
        return random.randrange(0, 3)

Overwriting sciccors_random_opponent.py


In [44]:
%%writefile rp_halftime_opponent.py

# Агент №6 не знающий про ножницы и логику,
# играющий половину времени камнем,
# половину бумагой
def rp_halftime_opponent(observation, configuration):
    #in case we have information about opponent last move
    half_time = observation.step/2
    if observation.step <= half_time:
        return 0
    #initial step
    else:
        return 1

Overwriting rp_halftime_opponent.py


In [45]:
%%writefile ps_halftime_opponent.py

# Агент № 7 играющий половину времени
# бумагой, а другую ножницами
def ps_halftime_opponent(observation, configuration):
    #in case we have information about opponent last move
    half_time = observation.step/2
    if observation.step <= half_time:
        return 1
    #initial step
    else:
        return 2

Overwriting ps_halftime_opponent.py


In [46]:
%%writefile rs_halftime_opponent.py

# Агент № 8 играющий половину времени
# камнем, а другую ножницами
def rs_halftime_opponent(observation, configuration):
    #in case we have information about opponent last move
    half_time = observation.step/2
    if observation.step <= half_time:
        return 0
    #initial step
    else:
        return 2

Overwriting rs_halftime_opponent.py


In [47]:
%%writefile even_number_opponent.py

# Агент №9 отвечающий на действия игрока на
# чётные числа раундов
def even_number_opponent(observation, configuration):
    #Начинает всегда с камня
    if observation.step == 0:
        return 0
    elif observation.step % 2 == 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Overwriting even_number_opponent.py


In [48]:
%%writefile rock_agent.py
#  Агент №10, всегда бросающий камень
def rock_agent_act(observation, configuration):
    return 0

Overwriting rock_agent.py


In [49]:
%%writefile statistical_agent.py
import random

# Агент №11, анализирующий статистику ходов оппонента
class StatisticalAgent():
    def __init__(self):
        self.action_histogram = {}

# Инициализируем экземпляр класса агента и метод его действия
agent = StatisticalAgent()
def act(observation, configuration):
    if observation.step == 0:
        agent.action_histogram = {}
        return

    # Агент реагирует на каждое предыдущее действие оппонента и записывает его в словарь action_histogram,
    # ключами которого являются действия оппонента, а значениями - количество тех или иных действий
    action = observation.lastOpponentAction
    if action not in agent.action_histogram:
        agent.action_histogram[action] = 0
    agent.action_histogram[action] += 1

    mode_action = None # действие оппонента, совершённое им наибольшее число раз
    mode_action_count = None # наибольшее количество действий оппонента

    for k, v in agent.action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v

    return (mode_action + 1) % configuration.signs

Overwriting statistical_agent.py


In [50]:
%%writefile copy_opponent_agent.py
import random

# Агент №12, копирующий предыдущий ход оппонента
def copy_opponent_agent_act(observation, configuration):
    # Если это не первый ход, выбросить значение предыдущего хода оппонента
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        # Если это первый ход, выбросить случайный элемент
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent_agent.py


Проведем соревнование между нашими агентами, чтобы оценить их эффективность. При формировании турнирных групп нужно учесть, что один и тот же агент не должен играть сам с собой. Для подсчета очков за раунд будем использовать функцию evaluate из библиотеки kaggle_environments. В ходе турнира будем вести статистику по двум показателям: общему количеству очков, набранных каждым агентом, и количеству его побед.

In [53]:
# Создаём словарь из экземпляров агентов
agents = {
    "paper": "Writing paper_agent.py",
    "scissors": "Writing scissors_agent.py",
    "rock": "Overwriting rock_agent.py",
    "paper_random": "Writing paper_random_opponent.py",
    "sciccors_random": "Writing sciccors_random_opponent.py",
    "rp_halftime": "Overwriting rp_halftime_opponent.py",
    "ps_halftime": "Writing ps_halftime_opponent.py",
    "rs_halftime": "Writing rs_halftime_opponent.py",
    "even_number": "Writing even_number_opponent.py",
    "copy_opponent": "Writing copy_opponent_agent.py",
    "statistical": "statistical_agent.py",
    "rock_random": "Writing rock_random_opponent.py",
}

# Инициализируем словари для результатов турнира
tournament_results = {}
tournament_wins = {}
# Инициализируем списки, в котором будут храниться результаты всех игр одного агента
tournament_table_row = []
tournament_wins_table_row = []

for i in agents.values():
    for j in agents.values():
        if i == j:
            round_result = 0 # игры с одинаковыми агентами не учитываются в турнире
        else:
           round_result = int(evaluate("rps", [i, j], configuration={"episodeSteps": 1000, "tieRewardThreshold": 10})[0][0])
        tournament_table_row += [round_result]
        tournament_wins_table_row += [0] if round_result <= 0 else [1]
    tournament_results[i] = tournament_table_row
    tournament_wins[i] = tournament_wins_table_row
    tournament_table_row = [] # очищаем списки для следующего агента
    tournament_wins_table_row = []

TypeError: '<' not supported between instances of 'str' and 'int'

In [54]:
# С помощью библиотеки pandas создадим таблицу результатов турнира
tournament_table = pd.DataFrame(tournament_results)
tournament_table.index = agents.keys() # устанавливаем индексы таблицы
tournament_table.columns = agents.keys()
tournament_table = tournament_table.T # транспонируем таблицу

# Дополнительно реализуем таблицу с общим числом побед каждого агента во всём турнире
tournament_wins_table = pd.DataFrame(tournament_wins)
tournament_wins_table.index = agents.keys()
tournament_wins_table.columns = agents.keys()
tournament_wins_table = tournament_wins_table.T

# Отобразим таблицу с помощью градиентной карты
tournament_table.style.background_gradient(cmap="Reds")

ValueError: Length mismatch: Expected axis has 0 elements, new values have 12 elements